이전에 작성한 글에서 프로그램은 결과를 내보이는 것이 목적이었습니다.

여기서는 결과가 아닌 과정(상태)을 내보이는 프로그램을 작성하겠습니다.

아래 코드를 보겠습니다.

### 1. Lazy Execution

In [1]:
def promise(x):
    yield x


def cons_stream(a, b):
    return (a, promise(b))

In [2]:
s0 = cons_stream(1, cons_stream(2, cons_stream(3, 4)))
s0

(1, <generator object promise at 0x104213678>)

위의 `tuple`이 가진 첫번째 원소인 `1`은 `s0`의 현재 상태입니다. 

그리고 두번째 원소는 `promise` 함수를 통해 값을 [제너레이터](https://wiki.python.org/moin/Generators)로 저장하고 있습니다.

즉, 이 `promise`는 다음 상태에 대한 __약속__일 뿐, 그에대한 계산을 하지 않습니다. 

( '약속'을 함으로써 다음 계산을 미루고 있기 때문에 이런 방법을 `Delayed execution` 또는 `Lazy execution`이라고 합니다. )

이처럼 스트림은 현재 상태를 가지고 다음 상태를 약속하는 차례열의(여기서는 `tuple`로 표현한) 연속입니다.



\* 원래 `SICP`에서는 '제너레이터'가 아닌 프로시저 실행을 지연함으로써 '약속'을 구현합니다.

간단히는, 아래처럼 구현할 수 있습니다.

```scheme
(define (delay proc)
    (lambda () proc))
```

적절한 함수를 만들어 `s0`를 한 번 쭉 뽑아보면,

In [3]:
def display_stream(s):
    if isinstance(s, tuple):
        print(s[0])
        return display_stream(next(s[1]))
    else:
        print(s)

In [4]:
display_stream(s0)

1
2
3
4


### 2. 스트림을 위한 인터페이스

다음으로, 

1. 임의의 정수 a부터 b까지($a < b$) 연속하는 스트림을 만들고,

2. 그 스트림에 특정 조건을 걸어서 필터링시키는 코드를 작성하겠습니다.

In [5]:
def stream_enumerate_interval(low, high):
    if high > low:
        return cons_stream(
            low, 
            stream_enumerate_interval(low+1, high))
    else:
        return high
    
    
def stream_filter(pred, stream):
    if not isinstance(stream, tuple):
        return stream
    elif pred(stream[0]):
        return cons_stream(
            stream[0],
            stream_filter(pred, next(stream[1])))
    else:
        return stream_filter(pred, next(stream[1]))

검증을 위해 1부터 10까지의 정수로 만들어진 스트림을 2로 나누어 떨어지는 경우만 필터링해보면,

In [6]:
s1 = stream_enumerate_interval(1, 10)
filtered_s1 = stream_filter(lambda x: x%2 == 0, s1)
display_stream(filtered_s1)

2
4
6
8
10


스트림을 위한 `map`함수를 만들면,

In [7]:
def stream_map(f, stream):
    if not isinstance(stream, tuple):
        return f(stream)
    else:
        return cons_stream(
            f(stream[0]),
            stream_map(f, next(stream[1]))
        )

`stream_map`이 여러 스트림을 받아들일 수 있도록 조금 더 개선해보겠습니다.

`stream_map` 함수는 1. 각 스트림의 현재상태를 묶은 후, 2. 거기에 필요한 연산을 수행한 결과를 3. 다시 스트림으로 만드는 기능을 합니다.

먼저, 1.의 경우인 각 스트림의 현재 상태를 묶기 위한 함수를 만들겠습니다.

스트림은 현재 상태와 다음 상태를 '약속'으로 가지고 있습니다. 

각 스트림의 0번째 원소는 현재상태를, 1번째 원소는 다음 상태의 약속을 나타냅니다.

따라서, 각 스트림의 n번째 원소를 추출해서 묶어주는( 여기서는 `tuple`로 ) 함수인  `nth_elements_extractor`을 만들어보면, 

```
nth_elements_extractor(sequences, index)
sequences : stream_map을 적용할 각 스트림을 이차원 tuple의 형태로 묶어 전달합니다.
index : 몇번째 인덱스의 원소들을 추출할지 입력합니다.
```
다음으로, 추출해서 묶은 데이터에 필요한 연산을 수행하는 함수를 만들겠습니다.

2장 [공통 인터페이스](http://wikibootup.github.io/sicp/2-1-conventional-interface.html)에서 만들었던 `accumulate`와 동일한 원리로 만들어주면 됩니다. 차이점은 초기값이 없다는 점입니다.

```
apply(op, sequence)
op : 필요한 연산
sequence : 데이터 묶음을 1차원 tuple 또는 list 형태로 전달
```


이제 `stream_map`을 개선하면,

In [23]:
def apply(op, sequence):
    if len(sequence) == 1:
        return sequence[0]
    else:
        return op(sequence[0], apply(op, sequence[1:]))
    
    
def nth_elements_extractor(index, sequences):
    if len(sequences) == 0:
        return ()
    else:
        return (sequences[0][index],) + \
            nth_elements_extractor(index, sequences[1:])


def stream_map(f, argstreams):
    if not isinstance(argstreams[0], tuple):
        return apply(f, argstreams)
    else:
        return cons_stream(
            apply(f, nth_elements_extractor(0, argstreams)),
            apply(stream_map, (
                    f, 
                    tuple(map(lambda e: next(e), 
                              nth_elements_extractor(1, argstreams))))))

검증을 위해 두 스트림의 각 원소끼리 더하는 연산을 수행하면,

In [24]:
from operator import add
s2 = stream_enumerate_interval(5, 15)
s3 = stream_enumerate_interval(10, 20)

a = stream_map(add, (s2, s3))
display_stream(a)

15
17
19
21
23
25
27
29
31
33
35


다음 번 글에 이어서 설명하도록 하겠습니다.